**SOW-MKI49: Neural Information Processing Systems**  
*Weeks 4 and 5: Assignment (225 points + 30 bonus points)*  
Author: Umut

In [ ]:
# Group number: ...
# Student 1 name, student 1 number: ...
# Student 2 name, student 2 number: ...
# Student 3 name, student 3 number: ...

In [1]:
import os
import numpy as np
import cv2
import keras
import pickle
import random
import pandas as pd
from keras.models import Sequential
from keras.layers import *
from keras import backend as k
from keras import optimizers
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.engine.topology import Layer
from keras_contrib.layers import CRF
from glob import glob
from time import time

Using TensorFlow backend.


**WaveNet component (75 points)**

* Implement missing parts of the call method (y and z). **25 points**
* Implement residual block class. **50 points**

---
Reminder:

* One convolution layer that has 61 kernels of size 2 with no nonlinearities.
![alt text](http://i67.tinypic.com/21mgi2w.png)
![alt text](http://i67.tinypic.com/292n04y.png)
---



In [2]:
root_dir = os.path.normpath(os.path.join(os.path.dirname(os.path.realpath('__file__'))))
data_directory = os.path.join(root_dir, 'lfw-piano_roll_data')

In [3]:
with open(os.path.join('piano_rolls.p'), 'rb') as f:
    piano_rolls = pickle.load(f)

In [4]:
keys = sorted(piano_rolls.keys())

random.seed(6)
random.shuffle(keys)

test_set = dict((key, piano_rolls[key]) for key in keys[:int(0.1 * len(keys))])
training_set = dict((key, piano_rolls[key]) for key in keys[int(0.1 * len(keys)):])
training_set_keys = list(training_set.keys())

In [5]:
#Tensor slicing function taken from https://github.com/keras-team/keras/issues/890#issuecomment-319671916

def crop(dimension, start, end):
    # Crops (or slices) a Tensor on a given dimension from start to end
    # example : to crop tensor x[:, :, 5:10]
    # call slice(2, 5, 10) as you want to crop on the second dimension
    def func(x):
        if dimension == 0:
            return x[start: end]
        if dimension == 1:
            return x[:, start: end]
        if dimension == 2:
            return x[:, :, start: end]
        if dimension == 3:
            return x[:, :, :, start: end]
        if dimension == 4:
            return x[:, :, :, :, start: end]
    return Lambda(func)

In [ ]:
def residual_block(y, i):
    
    short = y
    
    split1 = crop(4, 0, 61)(y)
    split2 = crop(4, 61, 122)(y)

    y1 = Conv2D(61, 1, dilation_rate = i)(split1)
    y2 = Conv2D(61, 1, dilation_rate = i)(split2)

    y1_tan = Activation('tanh')(y1)
    y2_sig = Activation('sigmoid')(y2)

    y_mul = Multiply()([y1_tan, y2_sig])
    
    parallel_conv1 = Conv2D(61, 1)(y_mul)
    
    y_ = Add()([short, parallel_conv1]) #(B)
        
    if i != 5:
        return Conv2D(512, 1)(y_mul)
    elif i == 5:
        return y_

In [ ]:
input_shape = (61, 80, 1)

batch_shape = (None,) + input_shape

input_ = k.placeholder(shape = batch_shape)

paddings = k.constant([[0, 0], [0, 0], [1, 0], [0, 0]])  # (batch_size, notes, time, channels)

padded_input = k.pad(input_, paddings, mode='CONSTANT', constant_values=0.0)

padded_shape = padded_input.shape

input1 = Input(padded_shape, batch_shape, tensor = padded_input)

model = Conv2D(61 * 2, (1, 2))(input1)

res1 = residual_block(model, 0)

res2 = residual_block(res1, 1)

res3 = residual_block(res2, 2)

res4 = residual_block(res3, 3)

res5 = residual_block(res4, 4)

res6 = residual_block(res5, 5)

model = Conv2D(3843, 1)(res6)



In [6]:
[2 ** (i % 6) for i in range(6)]

[1, 2, 4, 8, 16, 32]

In [ ]:
class _WaveNet(ChainList):
    def __init__(self):
        links = (L.DilatedConvolution2D(61, 2 * 61, (1, 2)),)
        
        links += tuple(_ResidualBlock((1, 2 ** (i % 6))) for i in range(6))
        links += (L.Convolution2D(512, 512, 1), L.Convolution2D(512, 3843, 1))

        super(_WaveNet, self).__init__(*links)

    def __call__(self, x):
        y = (self[0](F.pad(x, ((0, 0), (0, 0), (0, 0), (1, 0)), 'constant')),)
        z = 0
        h = F.split_axis(layer(x), 2, 1)
        y = F.sigmoid(h[0]) * F.tanh(h[1])

        for i in range(1, len(self) - 2):
            y = self[i]()
            #z +=

        #y, z =

        return F.reshape(y, (y.shape[0], 61, 61, y.shape[3])), \
               F.reshape(z, (z.shape[0], 2, 61, z.shape[3]))

class _ResidualBlock(ChainList):
    def __init__(self):
        
        with self.init_scope():
            self.convolution2D_0 = L.convolution2D(in_channels, 64, (1, 1))
            self.convolution2D_1 = L.convolution2D(in_channels, 512, 1)
            
            
    def __call__(self, x):
        
        
        
        
    pass

**CRF-RNN component (50 points)**

* Implement missing parts of the call method (z). **25 points**
* Why is z not normalized in the last iteration? **25 points**

---

Reminder:

![alt text](http://i68.tinypic.com/sy6mix.png)

---

In [ ]:
class _CRF(ChainList):
    def __init__(self):
        super(_CRF, self).__init__(L.ConvolutionND(1, 2, 2, 1, nobias = True))

    def __call__(self, x, y):
        #z =

        for i in range(5):
            #z =

            if i < 4:
                z = F.softmax(z)

        return z

**WaveCRF model (50 points)**

1. Implement missing parts of the call method (k, psi_u and Q_hat). **20 points**
2. Implement missing parts of the save and load methods (save and load model). **10 points**
3. Implement missing parts of the test and train methods (forward and/or backward propagate). **20 points**

In [ ]:
class WaveCRF(object):
    def __init__(self):
        self.log = {('test', 'accuracy'): (), ('test', 'loss'): (), ('training', 'accuracy'): (),
                    ('training', 'loss'): ()}
        self.model = ChainList(_WaveNet(), _CRF())
        self.optimizer = optimizers.Adam(0.0002, 0.5)

        self.optimizer.setup(self.model)

    def __call__(self, x):
        #k, psi_u =
        #Q_hat =

        return F.transpose(F.reshape(Q_hat, (x.shape[0], x.shape[3], 2, 61)), (0, 2, 3, 1))

    @classmethod
    def load(cls, directory):
        self = cls()
        self.log = np.load('{}/log.npy'.format(directory))

        # Load model
        serializers.load_npz('{}/optimizer.npz'.format(directory), self.optimizer)

        return self

    def save(self, directory):
        np.save('{}/log.npy'.format(directory), self.log)
        # Save model
        serializers.save_npz('{}/optimizer.npz'.format(directory), self.optimizer)

    def test(self, Q, x):
        with chainer.using_config('train', False):
            # Forward prop
            # Forward prop

            self.log['test', 'accuracy'] += (float(F.accuracy(Q_hat, Q).data),)
            self.log['test', 'loss'] += (float(loss.data),)

    def train(self, Q, x):
        # Forward prop
        # Forward prop

        # Backprop
        # Backprop
        # Backprop

        self.log['training', 'accuracy'] += (float(F.accuracy(Q_hat, Q).data),)
        self.log['training', 'loss'] += (float(loss.data),)

In [ ]:
%matplotlib inline

import IPython
import chainer
import matplotlib
import numpy
import os
import pickle
import random
import tqdm

In [ ]:
batch_size = 30
epochs = 100
root = '..'

In [ ]:
with open('{}/Data/piano_rolls.p'.format(root), 'rb') as f:
    piano_rolls = pickle.load(f)

keys = sorted(piano_rolls.keys())

random.seed(6)
random.shuffle(keys)

test_set = dict((key, piano_rolls[key]) for key in keys[:int(0.1 * len(keys))])
training_set = dict((key, piano_rolls[key]) for key in keys[int(0.1 * len(keys)):])
training_set_keys = list(training_set.keys())

In [ ]:
waveCRF = WaveCRF()

waveCRF.model.to_gpu()

In [ ]:
for epoch in tqdm.tnrange(epochs):
    random.shuffle(training_set_keys)

    batch = ()

    for key in tqdm.tqdm_notebook(training_set_keys, leave = False):
        i = random.randint(0, training_set[key].shape[1] - 80)
        batch += (training_set[key][32 : 93, i : i + 80],)

        if len(batch) == batch_size:
            batch = waveCRF.model.xp.array(batch)

            waveCRF.train(batch[:, :, 1:].astype('i'), batch[:, :, None, :-1].astype('f'))

            batch = ()

    for key in tqdm.tqdm_notebook(test_set, leave = False):
        batch = waveCRF.model.xp.array((test_set[key][32 : 93],))

        waveCRF.test(batch[:, :, 1:].astype('i'), batch[:, :, None, :-1].astype('f'))

    IPython.display.clear_output()

    for i, key in enumerate(waveCRF.log):
        matplotlib.pyplot.subplot(221 + i)
        matplotlib.pyplot.plot(numpy.array(waveCRF.log[key]).reshape(epoch + 1, -1).mean(1))
        matplotlib.pyplot.xlabel('iteration')
        matplotlib.pyplot.ylabel(key)

    matplotlib.pyplot.tight_layout()
    matplotlib.pyplot.show()
    os.makedirs('{}/Models/WaveCRF/{}'.format(root, epoch))
    waveCRF.save('{}/Models/WaveCRF/{}'.format(root, epoch))

**Test (50 points)**  

* Generate a number of samples, pick the best one and play it in the notebook. **50 points**

In [ ]:
# Test

**Bonus question (30 points)**

* Discuss how you can improve the model (you can talk about different architectures or different ways to encode the inputs, etc.) **10 points**
* Discuss the assumptions behind the meanfield approximation and its shortcomings. **10 points**
* Prove that the iterative update equation (CRF-RNN component) is differentiable so that we can backpropagate through them. **10 points**